# Classical Music classification using neural networks


In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, LSTM
import pickle 
import os
import librosa

/opt/anaconda3/envs/final/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/final/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/final/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/final/lib/python3.6/site-packages/tensorflow/python/f

In [2]:
SR=44100
sentence_length = 60
BATCH_SIZE=15
WORD_SIZE = 4 * SR
EPOCH = 3

In [3]:
def wav2sentences(nums, label):
  length = sentence_length * SR
  final = [nums[i * length:(i + 1) * length] for i in range((len(nums) + length - 1) // length )][:-1]
  if len(final) >= 1
    labels = [label for i in range(len(final))]
    final = np.array(final)
    final = final.reshape(final.shape[0], -1, WORD_SIZE)
    return final, labels
  return [], []

# target_path is the csv/pickle file that strores the trackID -> label

def process_data(src, target_path):
  X = []
  y = []
  with open(target_path, 'rb') as handle:
    id2label = pickle.load(handle)
  for root, dirs, files in os.walk(src):
    for file in files:
      x, sr = librosa.load(os.path.join(root, file), sr=SR)
      filename, file_ext = os.path.splitext(file)
      X_temp, y_temp = wav2sentences(x, id2label[int(filename)])
      for x in X_temp:
        X.append(x)
      for label in y_temp:
        y.append(label)
  d = {"data": X, "label": y}
  with open("data.pickle", 'wb') as handle:
    pickle.dump(d, handle)
process_data("./musicnet/wav", "./musicnet/musicnet_labels.pkl")

In [10]:
with open("./data.pickle", 'rb') as handle:
  d = pickle.load(handle)

X = d["data"]
y = d["label"]

num_label = len(set(y))
labels = list(set(y))
labels2ind = {}

for index in range(len(labels)):
  labels2ind.update({labels[index]: index}) 

y = [labels2ind[a] for a in y]

X = np.array(X)
y = np.array(y)

# Suppose we have a working X, and y, both 2D

X = X.reshape((-1, sentence_length // 4, WORD_SIZE))
# y = y.reshape((-1, )

X_train = X[:int(X.shape[0] * 0.8)]
X_test = X[int(X.shape[0] * 0.8):]

y_train = y[:int(X.shape[0] * 0.8)]
y_test = y[int(X.shape[0] * 0.8):]


In [1]:
LENGTH=3 * 44100
def getModel():
    model = Sequential()
    model.add(LSTM(1025, input_shape=(1025, 259), return_sequences=True))
    model.add(Flatten())
    model.add(Dense(num_label, activation='sigmoid'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

def train(model, train_generator, EPOCHS=20):
    model.fit_generator(generator=train_generator, epochs=EPOCHS)

def test(model, X_test, y_test):
    num_corr = 0
    for id, source in X_test.items():
        splitted = [X_test[id][i * length : (i + 1) * length] for i in range((len(X_test[id]) + length) // length)][:-1]
        label = y_test[id]
        prediction = model.predict(splitted)
        counts = np.bincount(prediction)
        predict_label = np.argmax(counts)
        if predict_label == label:
            num_corr += 1
    return (double)num_corr/len(X_test)



In [12]:

train_generator = MusicSentencesDataGenerator(train_ids, labels, "sample_data/", batch_size=20, SR=10)
# test_generator = MusicSentencesDataGenerator(test_ids, labels, "sample_data/", batch_size=20, SR=10)


model = get_model()
train(model, train_generator)



# scores = model.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 46.92%
